In [ ]:
import pickle as pkl

import polars as pl

from backtest_lib.market.polars_impl import PolarsPastView

close_df = pkl.load(open("./sp500_close.pkl", "rb"))
close_df = close_df.dropna(axis=1, how="any")
dates = close_df.index.values

close_pl = pl.from_pandas(close_df)
securities = close_pl.columns
close_prices_df = close_pl.with_columns(pl.Series("date", dates))
past_cost_prices = PolarsPastView.from_dataframe(close_prices_df)

past_cost_prices

In [ ]:
from backtest_lib.market import MarketView, PastUniversePrices
from backtest_lib.market.polars_impl import PolarsPastView

df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_past_view = PolarsPastView.from_dataframe(ewma)

market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"ewma_ratio": ewma_past_view},
)

market_view.signals["ewma_ratio"].by_period[-1]

In [ ]:
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.engine.decision import target_weights


def ewma_strategy(universe, current_portfolio, market, ctx):
    latest_ewma_ratio = (
        market.signals["ewma_ratio"].by_security[list(universe)].by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return target_weights(weights)


decision = ewma_strategy(["AAPL", "MSFT"], None, market_view, None)
decision

In [ ]:
import polars as pl

from backtest_lib.backtest import Backtest
from backtest_lib.market.polars_impl import PolarsUniverseMapping
from backtest_lib.portfolio import uniform_portfolio

universe = tuple(past_cost_prices.by_security.as_df(show_periods=False).columns)
pf = uniform_portfolio(universe, value=100000, backend="polars")
backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

In [ ]:
from backtest_lib.engine.decision import hold

hold_strategy = lambda *args, **kwargs: hold()


benchmark_backtest = Backtest(
    universe=universe,
    strategy=hold_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

results = benchmark_backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

In [ ]:
import polars as pl

first_prices = past_cost_prices.by_period[0]
last_prices = past_cost_prices.by_period[-1]

avg_first_change = (
    past_cost_prices.by_period[1] / past_cost_prices.by_period[0]
).mean()

print("avg_first_change:", avg_first_change)

(last_prices / first_prices).mean()

In [ ]:
aapl_prices = past_cost_prices.by_security["AAPL"].to_series()

print(aapl_prices.pct_change() + 1)
print((aapl_prices.pct_change() + 1).cum_prod())

In [ ]:
past_cost_prices.by_period[-1].mean()

In [ ]:
df = past_cost_prices.by_security.as_df(show_periods=False)
cols = df.columns
ewma_weights = (
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM").alias(col)) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)

ewma_weights_past_view = PolarsPastView.from_dataframe(ewma_weights)
market_view = MarketView(
    prices=PastUniversePrices(close=past_cost_prices),
    signals={"precomputed_weights": ewma_weights_past_view},
)


def fast_ewma_strategy(universe, current_portfolio, market, ctx):
    return target_weights(market_view.signals["precomputed_weights"].by_period[-1])


fast_ewma_backtest = Backtest(
    universe=universe,
    strategy=fast_ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

In [ ]:
results = fast_ewma_backtest.run()
results.annualized_return

In [ ]:
(
    df.with_columns(pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in cols)
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in cols)
    .select(cols)
    .with_columns(pl.mean_horizontal(cols).alias("avg_ratio"))
    .with_columns(((pl.col(col) / pl.col("avg_ratio")) / len(cols)) for col in cols)
    .select(cols)
    .with_columns(date=dates)
)